This notebook will be about applying some simple methods that are not too complicated.

We want to treat our imaginary end-users as business stakeholders who have a problem and need a short-term solution (day 1 answer)

Given this context, lets come up with some easy answers.

In [10]:
import pandas as pd

### Using current leaderboard ranking, recent form and historical rivalries

In [11]:
# Things to add:
# Goals scored and conceded by both home and away team
# Form of home and away team
# Betting odds

In [12]:
def adjusted_probs(home_odds, draw_odds, away_odds):

    total_probs = sum([(1/odds) * 100 for odds in [home_odds, draw_odds, away_odds]])

    return tuple(((1/odds) * 100 )/total_probs for odds in [home_odds, draw_odds, away_odds])

In [13]:
matches_df = (
    pd.read_csv('../data/N1.csv')
    .filter(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'B365H', 'B365D', 'B365A'])
    .rename(columns=str.lower)
    .rename(columns = {
        'b365h': 'odds_home', 
        'b365d': 'odds_draw', 
        'b365a': 'odds_away'
    })
    .assign(
        date=lambda x: pd.to_datetime(x['date'], format='%d/%m/%Y'),
        adj_home=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[0], axis=1),
        adj_draw=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[1], axis=1),
        adj_away=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[2], axis=1),
    )
)

In [14]:
matches_df.head()

,date,hometeam,awayteam,fthg,ftag,ftr,odds_home,odds_draw,odds_away,adj_home,adj_draw,adj_away
0,2023-08-11,Volendam,Vitesse,1,2,A,3.40,3.75,2.0,0.277264,0.251386,0.471349
1,2023-08-12,PSV Eindhoven,Utrecht,2,0,H,1.36,5.00,8.5,0.698324,0.189944,0.111732
2,2023-08-12,Heerenveen,Waalwijk,3,1,H,1.73,4.00,4.5,0.550375,0.238037,0.211588
3,2023-08-12,Ajax,Heracles,4,1,H,1.20,7.00,13.0,0.791304,0.135652,0.073043
4,2023-08-12,Zwolle,Sparta Rotterdam,1,2,A,2.60,3.40,2.7,0.366613,0.280351,0.353035


In [15]:
from collections import defaultdict
import numpy as np

df = matches_df

# Convert 'date' to datetime and sort the dataframe
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['date'], inplace=True)

# Reinitializing the team_stats dictionary to reset the stats
team_stats = defaultdict(lambda: {'points': 0, 'goals_for': 0, 'goals_against': 0, 'matches': []})

# Function to update team stats
def update_team_stats(team, goals_for, goals_against, result):
    team_stats[team]['goals_for'] += goals_for
    team_stats[team]['goals_against'] += goals_against
    team_stats[team]['matches'].append(result)
    if result == 'W':
        team_stats[team]['points'] += 3
    elif result == 'D':
        team_stats[team]['points'] += 1

# Function to calculate form (last 5 matches)
def calculate_form(matches):
    if len(matches) < 5:
        return ''.join(matches)
    else:
        return ''.join(matches[-5:])

# Adjusting the code to calculate features using only preceding results
new_data_preceding = []

for index, row in df.iterrows():
    home_team = row['hometeam']
    away_team = row['awayteam']
    home_goals = row['fthg']
    away_goals = row['ftag']
    result = row['ftr']

    # Prepare team rankings based on stats before this match
    teams_ranked = sorted(team_stats.keys(), key=lambda x: (team_stats[x]['points'], 
                                                            team_stats[x]['goals_for'] - team_stats[x]['goals_against'], 
                                                            team_stats[x]['goals_for']), 
                          reverse=True)

    # Append the new stats to the list
    new_row_data_preceding = {
        'home_position': teams_ranked.index(home_team) + 1 if home_team in team_stats else None,
        'away_position': teams_ranked.index(away_team) + 1 if away_team in team_stats else None,
        'home_points': team_stats[home_team]['points'] if home_team in team_stats else None,
        'away_points': team_stats[away_team]['points'] if away_team in team_stats else None,
        'home_goals_for': team_stats[home_team]['goals_for'] if home_team in team_stats else None,
        'away_goals_for': team_stats[away_team]['goals_for'] if away_team in team_stats else None,
        'home_goals_against': team_stats[home_team]['goals_against'] if home_team in team_stats else None,
        'away_goals_against': team_stats[away_team]['goals_against'] if away_team in team_stats else None,
        'home_form': calculate_form(team_stats[home_team]['matches']) if home_team in team_stats else None,
        'away_form': calculate_form(team_stats[away_team]['matches']) if away_team in team_stats else None
    }
    new_data_preceding.append(new_row_data_preceding)

    # Determine match result for home and away teams for updating stats after appending
    home_result = 'D' if result == 'D' else 'W' if result == 'H' else 'L'
    away_result = 'D' if result == 'D' else 'L' if result == 'H' else 'W'

    # Update team stats after recording the current stats
    update_team_stats(home_team, home_goals, away_goals, home_result)
    update_team_stats(away_team, away_goals, home_goals, away_result)

# Create a new DataFrame from the new data and concatenate it with the original DataFrame
new_df_preceding = pd.DataFrame(new_data_preceding)
result_df_preceding = pd.concat([df.reset_index(drop=True), new_df_preceding], axis=1)

In [16]:
def calculate_form_score(form):
    return form.count('W')*3 + form.count('D') if isinstance(form, str) else None

In [17]:
(
    result_df_preceding
    .assign(
        score_home_form = lambda df: df.apply(lambda row: calculate_form_score(row.home_form), axis = 1),
        score_away_form = lambda df: df.apply(lambda row: calculate_form_score(row.away_form), axis = 1),
        standings_difference = lambda df: abs(df.home_position - df.away_position),
        standings_sum = lambda df: df.home_position + df.away_position,
        relative_standings = lambda df: df.standings_sum / df.standings_difference,
        combined_form = lambda df: df.score_home_form  + df.score_away_form,
        combined_goals_for = lambda df: df.home_goals_for + df.away_goals_for,
        abs_diff_adj_odds=lambda x: abs(x.adj_home - x.adj_away),
        z_combined_form = lambda df: (df.combined_form - df.combined_form.mean()) / df.combined_form.std(),
        z_combined_goals_for = lambda df: (df.combined_goals_for - df.combined_goals_for.mean()) / df.combined_goals_for.std(),
        z_abs_diff_adj_odds = lambda df: (df.abs_diff_adj_odds - df.abs_diff_adj_odds.mean()) / df.abs_diff_adj_odds.std(),
        combined_score = lambda df: df.z_combined_form + df.z_combined_goals_for - df.z_abs_diff_adj_odds,
    )
    .query('date > "2023-12-06"')
    # .sort_values('combined_score', ascending = False)
    # .sort_values('abs_diff_adj_odds', ascending = True)
    # .sort_values('standings_sum', ascending = True)
    # .sort_values('z_combined_form', ascending = False)
    .sort_values('combined_form', ascending = False)
)

,date,hometeam,awayteam,fthg,ftag,ftr,odds_home,odds_draw,odds_away,adj_home,...,standings_difference,standings_sum,relative_standings,combined_form,combined_goals_for,abs_diff_adj_odds,z_combined_form,z_combined_goals_for,z_abs_diff_adj_odds,combined_score
127,2023-12-07,PSV Eindhoven,Heerenveen,2,0,H,1.18,7.00,13.0,0.794066,...,7.0,9.0,1.285714,27.0,73.0,0.721990,2.406617,2.962906,1.474417,3.895106
129,2023-12-09,Ajax,Sparta Rotterdam,2,1,H,1.40,4.75,7.5,0.675036,...,1.0,13.0,13.000000,22.0,50.0,0.549029,1.608213,1.549025,0.757618,2.399620
132,2023-12-10,Go Ahead Eagles,Utrecht,0,2,A,2.15,3.40,3.4,0.441558,...,8.0,20.0,2.500000,15.0,39.0,0.162338,0.490448,0.872820,-0.844942,2.208211
130,2023-12-09,Zwolle,Nijmegen,1,3,A,2.38,3.50,2.9,0.399890,...,3.0,21.0,7.000000,13.0,44.0,0.071704,0.171087,1.180186,-1.220553,2.571825
128,2023-12-08,Twente,Excelsior,4,2,H,1.22,6.50,12.0,0.775579,...,6.0,14.0,2.333333,11.0,52.0,0.696729,-0.148275,1.671971,1.369728,0.153967
133,2023-12-10,For Sittard,Waalwijk,1,0,H,1.91,3.60,4.0,0.497994,...,2.0,28.0,14.000000,11.0,29.0,0.260202,-0.148275,0.258089,-0.439364,0.549179
126,2023-12-07,Feyenoord,Volendam,3,1,H,1.06,12.00,26.0,0.885659,...,15.0,19.0,1.266667,10.0,56.0,0.849551,-0.307956,1.917863,2.003070,-0.393163
131,2023-12-09,AZ Alkmaar,Almere City,4,1,H,1.18,7.00,15.0,0.801772,...,12.0,20.0,1.666667,10.0,43.0,0.738699,-0.307956,1.118713,1.543665,-0.732908
134,2023-12-10,Vitesse,Heracles,2,0,H,1.95,3.60,3.8,0.486660,...,3.0,31.0,10.333333,4.0,30.0,0.236926,-1.266040,0.319563,-0.535825,-0.410653


### Now with an API

In [18]:
import requests

uri = 'https://api.football-data.org/v4/competitions/DED/matches'
headers = { 'X-Auth-Token': '5ee7f2b5ace94caf9f8668333873a90f' }

response = requests.get(uri, headers=headers)
data = response.json()['matches']

matches_df = pd.json_normalize(data)

In [19]:
cols_to_keep = [
    'utcDate', 'status', 'matchday', 'season.currentMatchday', 
    'homeTeam.tla', 'homeTeam.name', 'homeTeam.crest',
    'awayTeam.tla', 'awayTeam.name', 'awayTeam.crest',
    'score.winner', 'score.fullTime.home', 'score.fullTime.away'
]

matches_df = (
    matches_df
    .filter(cols_to_keep)
    .rename(columns = {
        'utcDate': 'date', 
        'season.currentMatchday':'currentMatchday',
        'homeTeam.tla':'home_team_tla',
        'homeTeam.name': 'home_team_name', 
        'homeTeam.crest': 'home_team_crest',
        'awayTeam.tla': 'away_team_tla', 
        'awayTeam.name': 'away_team_name', 
        'awayTeam.crest': 'away_team_crest',
        'score.winner': 'ftr', 
        'score.fullTime.home': 'fthg', 
        'score.fullTime.away': 'ftag'
    })
    .rename(columns=str.lower)
)

In [20]:
matches_df.query('matchday > 15 & matchday < 19')

,date,status,matchday,currentmatchday,home_team_tla,home_team_name,home_team_crest,away_team_tla,away_team_name,away_team_crest,ftr,fthg,ftag
135,2023-12-15T19:00:00Z,FINISHED,16,16,NEC,NEC,https://crests.football-data.org/1915.png,SIT,Fortuna Sittard,https://crests.football-data.org/1920.png,HOME_TEAM,4.0,1.0
136,2023-12-16T15:30:00Z,FINISHED,16,16,UTR,FC Utrecht,https://crests.football-data.org/676.png,RKC,RKC Waalwijk,https://crests.football-data.org/683.png,DRAW,1.0,1.0
137,2023-12-16T17:45:00Z,FINISHED,16,16,EXC,SBV Excelsior,https://crests.football-data.org/670.png,GOA,Go Ahead Eagles,https://crests.football-data.org/718.png,DRAW,1.0,1.0
138,2023-12-16T20:00:00Z,FINISHED,16,16,HEE,SC Heerenveen,https://crests.football-data.org/673.png,VOL,FC Volendam,https://crests.football-data.org/1919.png,AWAY_TEAM,1.0,2.0
139,2023-12-17T11:15:00Z,FINISHED,16,16,ALM,Almere City FC,https://crests.football-data.org/1911.png,VIT,SBV Vitesse,https://crests.football-data.org/679.png,HOME_TEAM,5.0,0.0
140,2023-12-17T13:30:00Z,FINISHED,16,16,SPA,Sparta Rotterdam,https://crests.football-data.org/6806.png,TWE,FC Twente '65,https://crests.football-data.org/666.png,DRAW,2.0,2.0
141,2023-12-17T13:30:00Z,FINISHED,16,16,HER,Heracles Almelo,https://crests.football-data.org/671.png,FEY,Feyenoord Rotterdam,https://crests.football-data.org/675.png,AWAY_TEAM,0.0,4.0
142,2023-12-17T15:45:00Z,FINISHED,16,16,AJA,AFC Ajax,https://crests.football-data.org/678.png,ZWO,PEC Zwolle,https://crests.football-data.org/684.png,DRAW,2.0,2.0
143,2023-12-17T19:00:00Z,FINISHED,16,16,AZ,AZ,https://crests.football-data.org/682.png,PSV,PSV,https://crests.football-data.org/674.png,AWAY_TEAM,0.0,4.0
144,2024-01-12T19:00:00Z,TIMED,17,16,RKC,RKC Waalwijk,https://crests.football-data.org/683.png,HER,Heracles Almelo,https://crests.football-data.org/671.png,None,NaN,NaN


In [21]:
matches_df.ftr.unique()

array(['AWAY_TEAM', 'HOME_TEAM', 'DRAW', None], dtype=object)

In [22]:
from collections import defaultdict

# Convert 'date' to datetime and sort the dataframe
matches_df['date'] = pd.to_datetime(matches_df['date'], utc = True)
matches_df.sort_values(by=['date'], inplace=True)

df = matches_df

# Reinitializing the team_stats dictionary to reset the stats
team_stats = defaultdict(lambda: {'points': 0, 'goals_for': 0, 'goals_against': 0, 'matches': []})

# Function to update team stats
def update_team_stats(team, goals_for, goals_against, result):
    team_stats[team]['goals_for'] += goals_for
    team_stats[team]['goals_against'] += goals_against
    team_stats[team]['matches'].append(result)
    if result == 'W':
        team_stats[team]['points'] += 3
    elif result == 'D':
        team_stats[team]['points'] += 1

# Function to calculate form (last 5 matches)
def calculate_form(matches):
    if len(matches) < 5:
        return ''.join(matches)
    else:
        return ''.join(matches[-5:])

# Adjusting the code to calculate features using only preceding results
new_data_preceding = []

for index, row in df.iterrows():
    if row['matchday'] <= row['currentmatchday'] + 1:

        home_team = row['home_team_name']
        away_team = row['away_team_name']
        home_goals = row['fthg']
        away_goals = row['ftag']
        result = row['ftr']

        # Prepare team rankings based on stats before this match
        teams_ranked = sorted(team_stats.keys(), key=lambda x: (team_stats[x]['points'], 
                                                                team_stats[x]['goals_for'] - team_stats[x]['goals_against'], 
                                                                team_stats[x]['goals_for']), 
                            reverse=True)

        # Append the new stats to the list
        new_row_data_preceding = {
            'home_position': teams_ranked.index(home_team) + 1 if home_team in team_stats else None,
            'away_position': teams_ranked.index(away_team) + 1 if away_team in team_stats else None,
            'home_points': team_stats[home_team]['points'] if home_team in team_stats else None,
            'away_points': team_stats[away_team]['points'] if away_team in team_stats else None,
            'home_goals_for': team_stats[home_team]['goals_for'] if home_team in team_stats else None,
            'away_goals_for': team_stats[away_team]['goals_for'] if away_team in team_stats else None,
            'home_goals_against': team_stats[home_team]['goals_against'] if home_team in team_stats else None,
            'away_goals_against': team_stats[away_team]['goals_against'] if away_team in team_stats else None,
            'home_form': calculate_form(team_stats[home_team]['matches']) if home_team in team_stats else None,
            'away_form': calculate_form(team_stats[away_team]['matches']) if away_team in team_stats else None
        }
        new_data_preceding.append(new_row_data_preceding)

        # Determine match result for home and away teams for updating stats after appending
        home_result = 'D' if result == 'DRAW' else 'W' if result == 'HOME_TEAM' else 'L'
        away_result = 'D' if result == 'DRAW' else 'L' if result == 'HOME_TEAM' else 'W'

        # Update team stats after recording the current stats
        update_team_stats(home_team, home_goals, away_goals, home_result)
        update_team_stats(away_team, away_goals, home_goals, away_result)

# Create a new DataFrame from the new data and concatenate it with the original DataFrame
new_df_preceding = pd.DataFrame(new_data_preceding)
result_df_preceding = pd.concat([df.reset_index(drop=True), new_df_preceding], axis=1)

In [23]:
from datetime import datetime
import pytz

# Ensure 'today' is timezone-aware (UTC)
utc = pytz.UTC
today = datetime.utcnow().replace(tzinfo=utc).strftime('%Y-%m-%d %H:%M:%S%z')

In [24]:
(
    result_df_preceding
    .query('matchday <= (currentmatchday + 1)')
    .query(f'date >= "{today}"')
    # .dropna(subset=['home_form', 'away_form'])
    .filter(['date', 'status', 'matchday', 'currentmatchday', 'home_team_tla', 'away_team_tla', 'fthg', 'ftag', 'home_form', 'away_form'])
    .assign(
        score_home_form = lambda df: df.apply(lambda row: calculate_form_score(row.home_form), axis = 1),
        score_away_form = lambda df: df.apply(lambda row: calculate_form_score(row.away_form), axis = 1),
        combined_form = lambda df: df.score_home_form  + df.score_away_form,
    )
    .sort_values('combined_form', ascending = False)
)

,date,status,matchday,currentmatchday,home_team_tla,away_team_tla,fthg,ftag,home_form,away_form,score_home_form,score_away_form,combined_form
152,2024-01-14 15:45:00+00:00,TIMED,17,16,FEY,NEC,NaN,NaN,WWLWW,DLWWW,12,10,22
148,2024-01-13 20:00:00+00:00,TIMED,17,16,PSV,EXC,NaN,NaN,WWWWW,DLDLD,15,3,18
150,2024-01-14 13:30:00+00:00,TIMED,17,16,GOA,AJA,NaN,NaN,WDLLD,WWWWD,5,13,18
145,2024-01-13 15:30:00+00:00,TIMED,17,16,SIT,SPA,NaN,NaN,WLWWL,WLWLD,9,7,16
146,2024-01-13 17:45:00+00:00,TIMED,17,16,TWE,AZ,NaN,NaN,DLWWD,WDLWL,8,7,15
147,2024-01-13 19:00:00+00:00,TIMED,17,16,ZWO,HEE,NaN,NaN,LLWLD,WWWLL,4,9,13
151,2024-01-14 13:30:00+00:00,TIMED,17,16,VIT,UTR,NaN,NaN,LLLWL,DWDWD,3,9,12
144,2024-01-12 19:00:00+00:00,TIMED,17,16,RKC,HER,NaN,NaN,WDLLD,LWLLL,5,3,8
149,2024-01-14 11:15:00+00:00,TIMED,17,16,VOL,ALM,NaN,NaN,LLLLW,DLLLW,3,4,7
